In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn as nn
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import copy
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T
import json

# Load model


In [2]:
'''
# Load mask rcnn model
def get_model(num_classes):
    pretrained_base_model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    # print(pretrained_base_model)

    in_features = pretrained_base_model.roi_heads.box_predictor.cls_score.in_features
    pretrained_base_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    mask_in_channels = pretrained_base_model.roi_heads.mask_predictor.conv5_mask.in_channels
    pretrained_base_model.roi_heads.mask_predictor = MaskRCNNPredictor(mask_in_channels, 256, num_classes)
    return pretrained_base_model

num_classes = 2  # the background class and the pedestrian class
maskrcnn = get_model(num_classes)
device = torch.device('cpu')
maskrcnn = torch.load("maskrcnn.pth", map_location=device)
maskrcnn = maskrcnn.to(device)
'''

'\n# Load mask rcnn model\ndef get_model(num_classes):\n    pretrained_base_model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)\n    # print(pretrained_base_model)\n\n    in_features = pretrained_base_model.roi_heads.box_predictor.cls_score.in_features\n    pretrained_base_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)\n\n    mask_in_channels = pretrained_base_model.roi_heads.mask_predictor.conv5_mask.in_channels\n    pretrained_base_model.roi_heads.mask_predictor = MaskRCNNPredictor(mask_in_channels, 256, num_classes)\n    return pretrained_base_model\n\nnum_classes = 2  # the background class and the pedestrian class\nmaskrcnn = get_model(num_classes)\ndevice = torch.device(\'cpu\')\nmaskrcnn = torch.load("maskrcnn.pth", map_location=device)\nmaskrcnn = maskrcnn.to(device)\n'

In [3]:
'''
# Load faster rcnn model
fasterrcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = fasterrcnn.roi_heads.box_predictor.cls_score.in_features
fasterrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
fasterrcnn.load_state_dict(torch.load("fasterrcnn_phase1.pth", map_location=device))
fasterrcnn = fasterrcnn.to(device)
'''

'\n# Load faster rcnn model\nfasterrcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)\nin_features = fasterrcnn.roi_heads.box_predictor.cls_score.in_features\nfasterrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)\nfasterrcnn.load_state_dict(torch.load("fasterrcnn_phase1.pth", map_location=device))\nfasterrcnn = fasterrcnn.to(device)\n'

In [4]:
# Load YOLO model
from ultralytics import YOLO
model_detection = YOLO('detect.pt')
model_segmentation = YOLO('segment.pt')

# Generate YOLO Prediction

In [5]:
def yolo_detect(input_directory, output_directory='output', detection_threshold=0.5):
    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Create "Count" folder in the output directory
    count_folder = os.path.join(output_directory, 'Count')
    os.makedirs(count_folder, exist_ok=True)

    bbox_path = os.path.join(output_directory, 'detection')
    os.makedirs(bbox_path, exist_ok=True)

    roi_folder = os.path.join(output_directory, 'ROI')
    os.makedirs(roi_folder, exist_ok=True)
    
    # List all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    for image_file in image_files:
        img_id = image_file.split('.')[0]
        image_path = os.path.join(input_directory, image_file)
        img = cv2.imread(image_path)
        img_copy = copy.deepcopy(img)
        results = model_detection(img, conf=detection_threshold)
        for r in results:
            bbox_list = r.boxes.xyxy
            count = len(bbox_list)  
            for i, bbox in enumerate(bbox_list):
                # Extract ROI
                roi = img_copy[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
                # Save ROI
                roi_path = os.path.join(roi_folder, f'{img_id}_{i+1}.jpg')
                cv2.imwrite(roi_path, roi)
                img_detect = cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 0, 255), 5)

            # Save image with bounding boxes
            detect_path = os.path.join(bbox_path, f'{img_id}_with_boxes.jpg')
            cv2.imwrite(detect_path, img_detect)

        # Save count
        count_filepath = os.path.join(count_folder, f'{img_id}.txt')
        with open(count_filepath, 'w') as f:
            f.write(str(count))

yolo_detect('input')


0: 704x1024 300 Maizes, 1235.3ms
Speed: 7.0ms preprocess, 1235.3ms inference, 3.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 300 Maizes, 1247.0ms
Speed: 7.5ms preprocess, 1247.0ms inference, 2.7ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 300 Maizes, 1219.7ms
Speed: 7.5ms preprocess, 1219.7ms inference, 3.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 300 Maizes, 1210.3ms
Speed: 6.3ms preprocess, 1210.3ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)


# Generate Faster RCNN Prediction

In [6]:
'''
# Generate faster RCNN prediction
def predict(model, images):
    model.eval()
    images = list(image.to(device) for image in images)
    outputs = model(images)
    return outputs

# Draw bounding box on processed image (1024x1024)
def draw_boxes_on_image(boxes, images):
    for box in boxes:
        cv2.rectangle(images,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (220, 0, 0), 3)
    return images

# Extract maize tassel image from bounding box
def extract_roi(img, img_id, boxes, output_directory):
    roi_folder = os.path.join(output_directory, 'ROI')
    os.makedirs(roi_folder, exist_ok=True)
    for i, box in enumerate(boxes):
        x1 = box[0]
        y1 = box[1]
        x2 = box[2]
        y2 = box[3]
        # Extract the region of interest (ROI)
        roi = img[y1:y2, x1:x2]

        # Save the ROI
        output_path = os.path.join(roi_folder, f'{img_id}_{i+1}.jpg')
        cv2.imwrite(output_path, roi)

# Resize bounding box coordinates to original image size
def resize_bbox(original_width, original_height, boxes):
    for box in boxes:

        # Extract coordinates
        x1 = box[0]
        y1 = box[1]
        x2 = box[2]
        y2 = box[3]

        # Calculate scale factor
        width_scale = original_width / 1024
        height_scale = original_height / 1024

        # Calculate new coordinates
        resized_x1 = int(x1 * width_scale)
        resized_y1 = int(y1 * height_scale)
        resized_x2 = int(x2 * width_scale)
        resized_y2 = int(y2 * height_scale)

        # Assign new coordinates
        box[0] = resized_x1
        box[1] = resized_y1
        box[2] = resized_x2
        box[3] = resized_y2
        
    return boxes

# faster rcnn inference
def process_images_and_predict(input_directory, output_directory='output', detection_threshold=0.6):
    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Create "Count" folder in the output directory
    count_folder = os.path.join(output_directory, 'Count')
    os.makedirs(count_folder, exist_ok=True)

    # List all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    for img_name in image_files:
        
        # Read and preprocess image
        img_id = img_name.split('.')[0]
        image_path = os.path.join(input_directory, img_name)
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (1024, 1024), cv2.INTER_AREA)
        img_res /= 255.0

        # Generate Faster RCNN prediction
        output = predict(fasterrcnn, [torch.tensor(img_res, dtype=torch.float32).permute(2, 0, 1).to(device)])
        prediction_boxes = output[0]['boxes'].data.cpu().numpy()
        scores = output[0]['scores'].data.cpu().numpy()
        count = len(prediction_boxes)
        # Filter boxes based on detection threshold
        prediction_boxes = prediction_boxes[scores >= detection_threshold].astype(np.int32)

        # Resize bounding boxes to original image size
        prediction_boxes_resized = resize_bbox(img.shape[1], img.shape[0], prediction_boxes)

        # Draw bounding box on image and save
        img_with_boxes = draw_boxes_on_image(prediction_boxes_resized, img_rgb)
        bbox_path = os.path.join(output_directory, 'detection')
        os.makedirs(bbox_path, exist_ok=True)
        output_path = os.path.join(bbox_path, f'{img_id}_with_boxes.jpg')
        cv2.imwrite(output_path, cv2.cvtColor(img_with_boxes, cv2.COLOR_RGB2BGR))

        # Run extract ROI for image
        extract_roi(img, img_id, prediction_boxes_resized, output_directory)

        # Save count to text file in "Count" folder
        count_filepath = os.path.join(count_folder, f'{img_id}.txt')
        with open(count_filepath, 'w') as f:
            f.write(str(count))

# Call the function with the input directory
input_directory = 'input'
process_images_and_predict(input_directory)
'''

'\n# Generate faster RCNN prediction\ndef predict(model, images):\n    model.eval()\n    images = list(image.to(device) for image in images)\n    outputs = model(images)\n    return outputs\n\n# Draw bounding box on processed image (1024x1024)\ndef draw_boxes_on_image(boxes, images):\n    for box in boxes:\n        cv2.rectangle(images,\n                      (box[0], box[1]),\n                      (box[2], box[3]),\n                      (220, 0, 0), 3)\n    return images\n\n# Extract maize tassel image from bounding box\ndef extract_roi(img, img_id, boxes, output_directory):\n    roi_folder = os.path.join(output_directory, \'ROI\')\n    os.makedirs(roi_folder, exist_ok=True)\n    for i, box in enumerate(boxes):\n        x1 = box[0]\n        y1 = box[1]\n        x2 = box[2]\n        y2 = box[3]\n        # Extract the region of interest (ROI)\n        roi = img[y1:y2, x1:x2]\n\n        # Save the ROI\n        output_path = os.path.join(roi_folder, f\'{img_id}_{i+1}.jpg\')\n        cv2

# Generate Mask RCNN Prediction

In [7]:
'''
from skimage.feature import graycomatrix
from skimage.measure import shannon_entropy
from skimage import io, color, util

def min_max_normalize_hu_moments(hu_moments):
    # Perform min-max normalization for a list of Hu moments
    min_value = min(hu_moments)
    max_value = max(hu_moments)

    normalized_hu_moments = [(value - min_value) / (max_value - min_value) for value in hu_moments]

    return normalized_hu_moments

# Color outliers
def detect_outliers(data, col_indices, lower_bound_multipliers, upper_bound_multipliers):
    outliers = np.zeros(len(data), dtype=bool)

    for col_index, lower_multiplier, upper_multiplier in zip(col_indices, lower_bound_multipliers, upper_bound_multipliers):
        q1 = np.percentile(data[:, col_index], 25)
        q3 = np.percentile(data[:, col_index], 75)
        iqr = q3 - q1
        lower_bound = q1 - lower_multiplier * iqr
        upper_bound = q3 + upper_multiplier * iqr

        if col_index == 2:
            # Skip upper bound comparison for data[:, 2]
            outliers_col = (data[:, col_index] < lower_bound)
        else:
            # Perform both lower and upper bound comparisons for other columns
            outliers_col = (data[:, col_index] < lower_bound) | (data[:, col_index] > upper_bound)

        outliers |= outliers_col

    return outliers

# Shape outliers
def find_outliers_combined_iqr(hu_moments_list):
    outliers = np.zeros(len(hu_moments_list), dtype=bool)
    # Calculate the Interquartile Range (IQR)
    q1 = np.percentile(hu_moments_list, 25)
    q3 = np.percentile(hu_moments_list, 75)
    iqr_value = q3 - q1

    # Define a multiplier to determine the outlier threshold
    iqr_multiplier = 1.5 

    # Define the lower and upper bounds for outliers
    lower_bound = q1 - iqr_multiplier * iqr_value
    upper_bound = q3 + iqr_multiplier * iqr_value

    # Identify outliers based on the bounds
    outliers_col = (hu_moments_list < lower_bound) | (hu_moments_list > upper_bound)
    outliers |= outliers_col

    return outliers

def process_images_in_directory(input_directory, model, device):

    # Get dominant color in HSV space
    def get_dominant_color_hsv(region):
        # Convert the region to HSV color space
        region_hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
        
        # Filter out black pixels
        non_black_pixels = (region_hsv[..., 2] != 0)
        region_without_black = region_hsv[non_black_pixels]

        # Calculate the mean color in HSV space
        dominant_color_hsv = np.mean(region_without_black, axis=0)
        
        return dominant_color_hsv
    
    # Set model to evaluation mode
    model.eval()
    result_list = []
    for filename in os.listdir(input_directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            
            # Read and process image
            img_path = os.path.join(input_directory, filename)
            img = Image.open(img_path)
            height, width = img.size
            resized_img = img.resize((256, 256))
            transform = T.ToTensor()
            img_tensor = transform(resized_img)

            # Get mask rcnn prediction
            with torch.no_grad():
                pred = model([img_tensor.to(device)])

            # Get mask output
            mask = (pred[0]["masks"][0].cpu().detach().numpy() * 255).astype("uint8").squeeze()

            # Resize mask
            resized_mask = cv2.resize(mask, (height, width))

            # Convert to binary mask
            resized_binary_mask = resized_mask > 100
            binary_mask = mask > 100
            # Get mask region
            img_numpy = np.array(img)
            binary_mask = binary_mask.astype(np.uint8)
            resized_binary_mask = resized_binary_mask.astype(np.uint8)
            masked_image = cv2.bitwise_and(img_numpy, img_numpy, mask=resized_binary_mask)
            # Get dominant color info
            dominant_color = get_dominant_color_hsv(masked_image)

            # Get image moments
            hu_moments = cv2.HuMoments(cv2.moments(binary_mask)).flatten()
            hu_moments_normalized = min_max_normalize_hu_moments(hu_moments)
            hu_sum = np.sum(hu_moments_normalized)
            # Save result
            result_list.append({
                'filename': filename,
                'dominant_color': dominant_color,
                'hu_moments': hu_moments_normalized,
                'hu_moments_sum': hu_sum,
            })

    # Detect color outliers
    dominant_colors = np.array([entry['dominant_color'] for entry in result_list])
    col_indices = [0, 1, 2]
    lower_multipliers = [1.5, 1.5, 1.25]
    upper_multipliers = [1.5, 1.5, 0]
    color_outliers = detect_outliers(dominant_colors, col_indices, lower_multipliers, upper_multipliers)

    # Mark color outliers in the result list
    for i in range(len(result_list)):
        result_list[i]['color_diff'] = str(color_outliers[i])
    
    # Detect shape outliers
    hu_moments_idx = [entry['hu_moments_sum'] for entry in result_list]
    shape_outliers = find_outliers_combined_iqr(hu_moments_idx)
    
    # Mark shape outliers in the result list
    for i in range(len(result_list)):
        result_list[i]['shape_diff'] = str(shape_outliers[i])

    # Save result list to JSON file
    output_directory = "output"  
    os.makedirs(output_directory, exist_ok=True)
    analysis_folder = os.path.join(output_directory, 'Analysis')
    os.makedirs(analysis_folder, exist_ok=True)

    for result in result_list:
        result['dominant_color'] = result['dominant_color'].tolist()
        filename = result['filename']
        json_filename = os.path.join(analysis_folder, f"{filename.split('.')[0]}.json")

        with open(json_filename, 'w') as json_file:
            json.dump(result, json_file, indent=4)


# Example use
roi_directory = "output/ROI"
process_images_in_directory(roi_directory, maskrcnn, device)
'''

'\nfrom skimage.feature import graycomatrix\nfrom skimage.measure import shannon_entropy\nfrom skimage import io, color, util\n\ndef min_max_normalize_hu_moments(hu_moments):\n    # Perform min-max normalization for a list of Hu moments\n    min_value = min(hu_moments)\n    max_value = max(hu_moments)\n\n    normalized_hu_moments = [(value - min_value) / (max_value - min_value) for value in hu_moments]\n\n    return normalized_hu_moments\n\n# Color outliers\ndef detect_outliers(data, col_indices, lower_bound_multipliers, upper_bound_multipliers):\n    outliers = np.zeros(len(data), dtype=bool)\n\n    for col_index, lower_multiplier, upper_multiplier in zip(col_indices, lower_bound_multipliers, upper_bound_multipliers):\n        q1 = np.percentile(data[:, col_index], 25)\n        q3 = np.percentile(data[:, col_index], 75)\n        iqr = q3 - q1\n        lower_bound = q1 - lower_multiplier * iqr\n        upper_bound = q3 + upper_multiplier * iqr\n\n        if col_index == 2:\n           

# Generate YOLO segmentation prediction

In [8]:
from matplotlib.patches import Polygon
def get_dominant_color_hsv(region):
        # Convert the region to HSV color space
        region_hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
        
        # Filter out black pixels
        non_black_pixels = (region_hsv[..., 2] != 0)
        region_without_black = region_hsv[non_black_pixels]

        # Calculate the mean color in HSV space
        dominant_color_hsv = np.mean(region_without_black, axis=0)
        
        return dominant_color_hsv

def min_max_normalize_hu_moments(hu_moments):
    # Perform min-max normalization for a list of Hu moments
    min_value = min(hu_moments)
    max_value = max(hu_moments)

    normalized_hu_moments = [(value - min_value) / (max_value - min_value) for value in hu_moments]

    return normalized_hu_moments

# Color outliers
def detect_outliers(data, col_indices, lower_bound_multipliers, upper_bound_multipliers):
    outliers = np.zeros(len(data), dtype=bool)

    for col_index, lower_multiplier, upper_multiplier in zip(col_indices, lower_bound_multipliers, upper_bound_multipliers):
        
        q1 = np.percentile(data[:, col_index], 25)
        q3 = np.percentile(data[:, col_index], 75)
        iqr = q3 - q1
        lower_bound = q1 - lower_multiplier * iqr
        upper_bound = q3 + upper_multiplier * iqr

        if col_index == 2:
            # Skip upper bound comparison for data[:, 2]
            outliers_col = (data[:, col_index] < lower_bound)
        else:
            # Perform both lower and upper bound comparisons for other columns
            outliers_col = (data[:, col_index] < lower_bound) | (data[:, col_index] > upper_bound)

        outliers |= outliers_col

    return outliers

# Shape outliers
def find_outliers_combined_iqr(hu_moments_list):
    outliers = np.zeros(len(hu_moments_list), dtype=bool)
    # Calculate the Interquartile Range (IQR)
    q1 = np.percentile(hu_moments_list, 25)
    q3 = np.percentile(hu_moments_list, 75)
    iqr_value = q3 - q1

    # Define a multiplier to determine the outlier threshold
    iqr_multiplier = 2.5 

    # Define the lower and upper bounds for outliers
    lower_bound = q1 - iqr_multiplier * iqr_value
    upper_bound = q3 + iqr_multiplier * iqr_value

    # Identify outliers based on the bounds
    outliers_col = (hu_moments_list < lower_bound) | (hu_moments_list > upper_bound)
    outliers |= outliers_col

    return outliers

def yolo_segment(input_directory, output_directory='output'):
    result_list = []
    # List all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    for image_file in image_files:
        img_id = image_file.split('.')[0]
        image_path = os.path.join(input_directory, image_file)
        img = cv2.imread(image_path)
        height, width, _ = img.shape
        results = model_segmentation(img)
        for r in results:
            # Create a blank image
            binary_mask = np.zeros((height, width), dtype=np.uint8)

            # No mask detected
            if not r:
                continue
            
            # Get mask
            mask = r.masks.xy

            # Draw the mask on the blank image
            for points in mask:
                # Convert the points to integer and reshape to (num_points, 1, 2)
                points = points.astype(int).reshape((-1, 1, 2))
                
                # Fill the polygon in the blank image
                cv2.fillPoly(binary_mask, [points], color=255)

            
            # Cut mask region
            img_numpy = np.array(img)
            masked_image = cv2.bitwise_and(img_numpy, img_numpy, mask=binary_mask)
            masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

            # Get dominant color info
            dominant_color = get_dominant_color_hsv(masked_image)
            
            # Get image moments
            hu_moments = cv2.HuMoments(cv2.moments(binary_mask)).flatten()
            hu_moments_normalized = min_max_normalize_hu_moments(hu_moments)
            hu_sum = np.sum(hu_moments_normalized)
            # Save result
            result_list.append({
                'filename': image_file,
                'dominant_color': dominant_color,
                'hu_moments': hu_moments_normalized,
                'hu_moments_sum': hu_sum,
            })
    # Detect color outliers
    # Todo: compare color range instead of IQR
    dominant_colors = np.array([entry['dominant_color'] for entry in result_list])
    col_indices = [0, 1, 2]
    lower_multipliers = [3, 3, 2]
    upper_multipliers = [3, 3, 0]
    color_outliers = detect_outliers(dominant_colors, col_indices, lower_multipliers, upper_multipliers)

    # Detect shape outliers
    hu_moments_idx = [entry['hu_moments_sum'] for entry in result_list]
    shape_outliers = find_outliers_combined_iqr(hu_moments_idx)

    # Save result list to JSON file for color outliers
    output_directory = "output"  
    os.makedirs(output_directory, exist_ok=True)
    analysis_folder = os.path.join(output_directory, 'Analysis')
    os.makedirs(analysis_folder, exist_ok=True)

    for i in range(len(result_list)):
        if color_outliers[i] or shape_outliers[i]:
            result_list[i]['dominant_color'] = result_list[i]['dominant_color'].tolist()
            result_list[i]['color_diff'] = str(color_outliers[i])
            result_list[i]['shape_diff'] = str(shape_outliers[i])

            filename = result_list[i]['filename']
            json_filename = os.path.join(analysis_folder, f"{filename.split('.')[0]}.json")

            with open(json_filename, 'w') as json_file:
                json.dump(result_list[i], json_file, indent=4)
        

yolo_segment('output/ROI')


0: 256x256 7 tassels, 167.5ms
Speed: 1.0ms preprocess, 167.5ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 8 tassels, 145.4ms
Speed: 1.0ms preprocess, 145.4ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 6 tassels, 123.4ms
Speed: 1.5ms preprocess, 123.4ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 7 tassels, 118.4ms
Speed: 0.0ms preprocess, 118.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 3 tassels, 167.4ms
Speed: 0.0ms preprocess, 167.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 3 tassels, 116.8ms
Speed: 6.5ms preprocess, 116.8ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 7 tassels, 113.8ms
Speed: 1.0ms preprocess, 113.8ms inference, 4.5ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 9 tassels, 123.6ms
Speed: 1.0ms preprocess, 123.6ms inference, 5.5ms postprocess per 